In [1]:
from pathlib import Path
import numpy as np
# import h5py
from tqdm.auto import tqdm, trange
import scipy.io
import time
import torch
import random
import math
import shutil
from sklearn.decomposition import PCA
import scipy as sp
import scipy.signal
import os
import pandas as pd
from torch import nn
from scipy.spatial.distance import cdist
# import spikeinterface.core as sc
# import spikeinterface.full as si

from analysis.projections import pca_train, pca, get_flattened_data
from analysis.plotting import plot_gmm, plot_closest_spikes
from analysis.encoder_utils import load_GPT_backbone, get_fcenc_backbone
from analysis.cluster import GMM, HDBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from analysis.cluster import MeanShift
from analysis.benchmarking import class_scores, avg_score, per_class_accs, avg_class_accs
import matplotlib.patheffects as pe

from ceed.models.model_simclr import FullyConnectedEnc
from utils.utils import gmm_monitor, knn_monitor
from data_aug.wf_data_augs import Crop
from data_aug.contrastive_learning_dataset import WFDataset_lab 

/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be 

In [2]:
# class Args(Object):
#     def __init__(epoch_nums=1):
#         epochs = epoch_nums
    
class Args(object):
    pass

args = Args()
args.epochs = 1
args.use_chan_pos = False
args.use_gpt = False
print(args.epochs)

1


In [3]:
ten_neur_id_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_random_choice_fix_mc'
ten_neur_ood_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_10_neuron_400ood'
ten_neur_ood_path_dy009 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy009_10_neuron_400ood'
fourhund_neur_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real400n_200s'
fourhund_neur_1200_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real400n_1200s'
threenine_neur_1200_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real390n_1200s'
sixhund_neur_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real600n_1200s'
test_setA = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_og_goodunit_subset'
test_fn = 'spikes_test.npy' 
train_fn = 'spikes_train.npy'

single_chan_ckpt = '/Users/ankit/Documents/PaninskiLab/spike-psvae/pretrained/single_chan_denoiser.pt'

In [4]:
# dy016 ID train data
dy016_id_5c_train_dataset = WFDataset_lab(ten_neur_id_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy016_id_5c_train_data, dy016_id_5c_train_labels = get_flattened_data(dy016_id_5c_train_dataset)
dy016_denoised_id_5c_train_data, dy016_denoised_id_5c_train_labels = get_flattened_data(dy016_id_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy016_id_11c_train_dataset = WFDataset_lab(ten_neur_id_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy016_id_11c_train_data, dy016_id_11c_train_labels = get_flattened_data(dy016_id_11c_train_dataset)
dy016_denoised_id_11c_train_data, dy016_denoised_id_11c_train_labels = get_flattened_data(dy016_id_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 400 neuron, 200 spikes train data
# fourhund_5c_train_dataset = WFDataset_lab(fourhund_neur_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# fourhund_5c_train_data, fourhund_5c_train_labels = get_flattened_data(fourhund_5c_train_dataset)
# fourhund_denoised_5c_train_data, fourhund_denoised_5c_train_labels = get_flattened_data(fourhund_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# fourhund_11c_train_dataset = WFDataset_lab(fourhund_neur_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# fourhund_11c_train_data, fourhund_11c_train_labels = get_flattened_data(fourhund_11c_train_dataset)
# fourhund_denoised_11c_train_data, fourhund_denoised_11c_train_labels = get_flattened_data(fourhund_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 400 neuron, 1200 spikes train data
# fourhund_1200_5c_train_dataset = WFDataset_lab(fourhund_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# fourhund_1200_5c_train_data, fourhund_1200_5c_train_labels = get_flattened_data(fourhund_1200_5c_train_dataset)
# fourhund_1200_denoised_5c_train_data, fourhund_1200_denoised_5c_train_labels = get_flattened_data(fourhund_1200_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# fourhund_1200_11c_train_dataset = WFDataset_lab(fourhund_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# fourhund_1200_11c_train_data, fourhund_1200_11c_train_labels = get_flattened_data(fourhund_1200_11c_train_dataset)
# fourhund_1200_denoised_11c_train_data, fourhund_1200_denoised_11c_train_labels = get_flattened_data(fourhund_1200_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 390 neuron, 1200 spikes train data
threenine_1200_5c_train_dataset = WFDataset_lab(threenine_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
threenine_1200_5c_train_data, threenine_1200_5c_train_labels = get_flattened_data(threenine_1200_5c_train_dataset)
threenine_1200_denoised_5c_train_data, threenine_1200_denoised_5c_train_labels = get_flattened_data(threenine_1200_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

threenine_1200_11c_train_dataset = WFDataset_lab(threenine_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
threenine_1200_11c_train_data, threenine_1200_11c_train_labels = get_flattened_data(threenine_1200_11c_train_dataset)
threenine_1200_denoised_11c_train_data, threenine_1200_denoised_11c_train_labels = get_flattened_data(threenine_1200_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016 ood data loaders
dy016_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_5c_memory_data, dy016_ood_5c_memory_labels = get_flattened_data(dy016_ood_5c_memory_dataset)
dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_memory_labels = get_flattened_data(dy016_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy016_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_5c_test_data, dy016_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)
dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy016_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_11c_memory_data, dy016_ood_11c_memory_labels = get_flattened_data(dy016_ood_11c_memory_dataset)
dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_memory_labels = get_flattened_data(dy016_ood_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy016_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_11c_test_data, dy016_ood_11c_test_labels = get_flattened_data(dy016_ood_11c_test_dataset)
dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels = get_flattened_data(dy016_ood_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy009 ood data loaders
dy009_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_5c_memory_data, dy009_ood_5c_memory_labels = get_flattened_data(dy009_ood_5c_memory_dataset)
dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_memory_labels = get_flattened_data(dy009_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy009_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_5c_test_data, dy009_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)
dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels = get_flattened_data(dy009_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy009_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_11c_memory_data, dy009_ood_11c_memory_labels = get_flattened_data(dy009_ood_11c_memory_dataset)
dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_memory_labels = get_flattened_data(dy009_ood_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

dy009_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_11c_test_data, dy009_ood_11c_test_labels = get_flattened_data(dy009_ood_11c_test_dataset)
dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels = get_flattened_data(dy009_ood_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# ten neuron ID data loaders
# dy009 ood data loaders
id_5c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
id_5c_memory_data, id_5c_memory_labels = get_flattened_data(id_5c_memory_dataset)
id_denoised_5c_memory_data, id_denoised_5c_memory_labels = get_flattened_data(id_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

id_5c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
id_5c_test_data, id_5c_test_labels = get_flattened_data(id_5c_test_dataset)
id_denoised_5c_test_data, id_denoised_5c_test_labels = get_flattened_data(id_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

id_11c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
id_11c_memory_data, id_11c_memory_labels = get_flattened_data(id_11c_memory_dataset)
id_denoised_11c_memory_data, id_denoised_11c_memory_labels = get_flattened_data(id_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

id_11c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
id_11c_test_data, id_11c_test_labels = get_flattened_data(id_11c_test_dataset)
id_denoised_11c_test_data, id_denoised_11c_test_labels = get_flattened_data(id_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

True spikes_train.npy
(12000, 21, 121)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12000/12000 [00:05<00:00, 2069.42it/s]


True spikes_train.npy
(12000, 21, 121)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12000/12000 [00:12<00:00, 999.21it/s]


True spikes_train.npy
(467004, 21, 121)


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 2053.07it/s]


True spikes_train.npy
(2000, 21, 121)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 955.41it/s]


(2000, 21, 121)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 984.94it/s]


In [5]:
def pca_gmm_average_tr(train_dataset, memory_dataset, test_dataset, test_labels, num_classes=10, num_iters=50, out_dim=5):
    scores = []
    for i in range(num_iters):
        # covariance_type : {'full', 'tied', 'diag', 'spherical'}
        covariance_type = 'full'
        reps_train, _, pca_ = pca(train_dataset, out_dim)
        reps_memory = pca_.transform(memory_dataset)
        reps_test = pca_.transform(test_dataset)
        gmm = GaussianMixture(num_classes,
                            random_state=random.randint(0, 1000000),
                            covariance_type=covariance_type).fit(reps_memory)
        gmm_cont_test_labels = gmm.predict(reps_test)
        curr_score = adjusted_rand_score(test_labels, gmm_cont_test_labels)*100
        scores.append(curr_score)
        if i == 49:
           print("max gmm score: {}".format(max(scores)))
           print("min gmm score: {}".format(min(scores)))
           print("50 run gmm mean score: {}".format(np.mean(scores)))
           print("50 run gmm std-dev score: {}".format(np.std(scores)))

    score = np.mean(scores)

    return score

def pca_gmm_average(memory_dataset, test_dataset, test_labels, num_classes=10, num_iters=50, out_dim=5):
    scores = []
    for i in range(num_iters):
        # covariance_type : {'full', 'tied', 'diag', 'spherical'}
        covariance_type = 'full'
        reps_train, _, pca_ = pca(memory_dataset, out_dim)
        reps_test = pca_.transform(test_dataset)
        gmm = GaussianMixture(num_classes,
                            random_state=random.randint(0, 1000000),
                            covariance_type=covariance_type).fit(reps_train)
        gmm_cont_test_labels = gmm.predict(reps_test)
        curr_score = adjusted_rand_score(test_labels, gmm_cont_test_labels)*100
        scores.append(curr_score)
        if i == 49:
           print("max gmm score: {}".format(max(scores)))
           print("min gmm score: {}".format(min(scores)))
           print("50 run gmm mean score: {}".format(np.mean(scores)))
           print("50 run gmm std-dev score: {}".format(np.std(scores)))

    score = np.mean(scores)

    return score

# New Neuron metrics

## 10 neuron, 1200 spikes, 5 channels

In [16]:
dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 49.94842592385938
min gmm score: 42.4561070568621
50 run gmm mean score: 46.41453202384358
50 run gmm std-dev score: 1.641824642630016
DY016 ID (200 train spikes) denoised data GMM score: 46.41453202384358
max gmm score: 43.23980993019478
min gmm score: 36.045935604862564
50 run gmm mean score: 38.97439442092881
50 run gmm std-dev score: 1.319551055255417
DY009 ID (200 train spikes) denoised data GMM score: 38.97439442092881
max gmm score: 55.71084489678194
min gmm score: 33.0873169073262
50 run gmm mean score: 46.00936104828743
50 run gmm std-dev score: 6.792019176866088
test set A (DY016) (200 train spikes) ID denoised data GMM score: 46.00936104828743


In [18]:
dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_id_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 40.260985988403114
min gmm score: 29.225243863254796
50 run gmm mean score: 34.60790511126595
50 run gmm std-dev score: 2.54071928339712
DY016 OOD (10 neurons, 1200 train spikes) data GMM score: 34.60790511126595
max gmm score: 26.317805234862252
min gmm score: 18.433767303204288
50 run gmm mean score: 21.828724358383383
50 run gmm std-dev score: 2.094112767631093
DY009 OOD (10 neurons, 1200 train spikes) data GMM score: 21.828724358383383
max gmm score: 51.152363011948665
min gmm score: 32.68867536638837
50 run gmm mean score: 43.49786698028805
50 run gmm std-dev score: 5.243246579351943
test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: 43.49786698028805


## 10 neuron, 1200 spikes, 11 channels

In [ ]:
dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

In [ ]:
dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

## 400 neuron, 200 spikes, 5 chan

In [24]:
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.852660255138034
min gmm score: 42.61961320461751
50 run gmm mean score: 45.859394513078385
50 run gmm std-dev score: 1.6699787348189143
DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: 45.859394513078385
max gmm score: 43.50847543736153
min gmm score: 35.619292892656055
50 run gmm mean score: 39.15385726020336
50 run gmm std-dev score: 2.1699930947155828
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 39.15385726020336
max gmm score: 54.64599017617606
min gmm score: 38.370586128739106
50 run gmm mean score: 49.26934635422893
50 run gmm std-dev score: 4.472748220905369
test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: 49.26934635422893


In [25]:
dy016_ood_score = pca_gmm_average_tr(fourhund_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 42.5790105809703
min gmm score: 24.839751754514143
50 run gmm mean score: 34.62326199084467
50 run gmm std-dev score: 4.1181057189157615
DY016 OOD (400 neurons, 200 train spikes) data GMM score: 34.62326199084467
max gmm score: 23.64360815251711
min gmm score: 15.598370364436695
50 run gmm mean score: 19.737513415238556
50 run gmm std-dev score: 1.5803748763485617
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 19.737513415238556
max gmm score: 48.41062989522548
min gmm score: 36.001991663148324
50 run gmm mean score: 43.56754906865924
50 run gmm std-dev score: 3.468305734304177
test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: 43.56754906865924


## 400 neuron, 200 spikes, 11 chan

In [26]:
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.244958295217174
min gmm score: 40.4848033111489
50 run gmm mean score: 46.448484524812066
50 run gmm std-dev score: 1.4455020400279521
DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: 46.448484524812066
max gmm score: 45.47727678049991
min gmm score: 38.70774512786305
50 run gmm mean score: 42.776622635266406
50 run gmm std-dev score: 1.8247957739373344
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 42.776622635266406
max gmm score: 54.28607334870959
min gmm score: 34.691665530745844
50 run gmm mean score: 42.675400186749386
50 run gmm std-dev score: 3.2538126642336005
test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: 42.675400186749386


In [27]:
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 41.60675125017051
min gmm score: 30.786828103802954
50 run gmm mean score: 36.10275655248079
50 run gmm std-dev score: 2.521434748232127
DY016 OOD (400 neurons, 200 train spikes) data GMM score: 36.10275655248079
max gmm score: 37.55857981940917
min gmm score: 27.450000211494384
50 run gmm mean score: 31.2089024732494
50 run gmm std-dev score: 2.53233762744236
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 31.2089024732494
max gmm score: 44.705760996409985
min gmm score: 33.909147579717605
50 run gmm mean score: 40.77009849126654
50 run gmm std-dev score: 3.3294820050708007
test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: 40.77009849126654


## 400 neuron, 1200 spikes, 5 chan

In [28]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.094395698234464
min gmm score: 41.745284448139024
50 run gmm mean score: 44.502718507091146
50 run gmm std-dev score: 1.7779936129471667
DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 44.502718507091146
max gmm score: 44.52775351653091
min gmm score: 34.04123445104575
50 run gmm mean score: 40.33534892495606
50 run gmm std-dev score: 2.6011016533929294
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 40.33534892495606
max gmm score: 55.92005422904447
min gmm score: 39.23059834144118
50 run gmm mean score: 48.38147559926359
50 run gmm std-dev score: 4.251579451897838
test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: 48.38147559926359


In [29]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 39.89104024339202
min gmm score: 26.546702953463004
50 run gmm mean score: 34.43533178154106
50 run gmm std-dev score: 3.5552293564893485
DY016 OOD (400 neurons, 1200 train spikes) data GMM score: 34.43533178154106
max gmm score: 26.50103263323565
min gmm score: 17.536011558451317
50 run gmm mean score: 20.862068292899586
50 run gmm std-dev score: 1.5991941748639513
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 20.862068292899586
max gmm score: 51.654971611343306
min gmm score: 35.082526937637624
50 run gmm mean score: 44.87747682661522
50 run gmm std-dev score: 4.001206501321743
test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: 44.87747682661522


## 400 neuron, 1200 spikes, 11 chan

In [30]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.99614025165182
min gmm score: 40.78003377881754
50 run gmm mean score: 46.727223066562104
50 run gmm std-dev score: 1.5269670513611084
DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 46.727223066562104
max gmm score: 45.14165294730949
min gmm score: 35.96376483977253
50 run gmm mean score: 41.27244488979042
50 run gmm std-dev score: 2.310532488437097
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 41.27244488979042
max gmm score: 55.17556987256861
min gmm score: 34.355718816344115
50 run gmm mean score: 44.6044794271138
50 run gmm std-dev score: 4.258011368895061
test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: 44.6044794271138


In [31]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 40.8809364902398
min gmm score: 31.559800924748416
50 run gmm mean score: 36.01833361293192
50 run gmm std-dev score: 2.358189822154822
DY016 OOD (400 neurons, 1200 train spikes) data GMM score: 36.01833361293192
max gmm score: 38.4876234446748
min gmm score: 27.708631557995485
50 run gmm mean score: 30.764194111764947
50 run gmm std-dev score: 2.1523340052588593
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 30.764194111764947
max gmm score: 43.2844852786714
min gmm score: 26.144529981858213
50 run gmm mean score: 37.77904473049833
50 run gmm std-dev score: 3.780511526098917
test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: 37.77904473049833


## 390 neuron, 1200 spikes, 11 chan

In [6]:
dy016_ood_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.60804477338698
min gmm score: 39.75480683937004
50 run gmm mean score: 45.92914436746882
50 run gmm std-dev score: 1.6328892611301953
DY016 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 45.92914436746882
max gmm score: 46.13402195125406
min gmm score: 35.80291780581771
50 run gmm mean score: 42.379070562153146
50 run gmm std-dev score: 2.2303673248404534
DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 42.379070562153146
max gmm score: 55.18234671845307
min gmm score: 35.598123840937454
50 run gmm mean score: 45.77485420791682
50 run gmm std-dev score: 4.1986008009968385
test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: 45.77485420791682


In [7]:
dy016_ood_score = pca_gmm_average_tr(threenine_1200_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (390 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(threenine_1200_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: " + str(testset_A_score))

max gmm score: 39.189555871252686
min gmm score: 28.935972677835114
50 run gmm mean score: 34.53771812976084
50 run gmm std-dev score: 2.1985301275746054
DY016 OOD (390 neurons, 1200 train spikes) data GMM score: 34.53771812976084
max gmm score: 36.17186157242666
min gmm score: 27.277896826425675
50 run gmm mean score: 31.420735723586652
50 run gmm std-dev score: 2.175408313716692
DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 31.420735723586652
max gmm score: 43.24709337992005
min gmm score: 32.93959869314438
50 run gmm mean score: 37.9835405610387
50 run gmm std-dev score: 3.344276432386825
test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: 37.9835405610387


## ID of all sets

In [9]:
dy016_ood_score = pca_gmm_average(dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 ID (200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 ID (200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (DY016) (200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 52.584032309117376
min gmm score: 46.0081820452671
50 run gmm mean score: 49.2891235247913
50 run gmm std-dev score: 1.6740202597724116
DY016 ID (200 train spikes) denoised data GMM score: 49.2891235247913
max gmm score: 47.609536052231974
min gmm score: 38.36627362684138
50 run gmm mean score: 43.84671865114465
50 run gmm std-dev score: 2.4790120498438704
DY009 ID (200 train spikes) denoised data GMM score: 43.84671865114465
max gmm score: 55.13788604316496
min gmm score: 41.24540178404644
50 run gmm mean score: 50.089815808558384
50 run gmm std-dev score: 3.7784439320488157
test set A (DY016) (200 train spikes) ID denoised data GMM score: 50.089815808558384


In [10]:
dy016_ood_score = pca_gmm_average(dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 ID (200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_labels)
print("DY009 ID (200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016) (200 train spikes) ID data denoised GMM score: " + str(testset_A_score))

max gmm score: 52.87893863727135
min gmm score: 46.19428913641999
50 run gmm mean score: 49.9818882433913
50 run gmm std-dev score: 1.3541427496136573
DY016 ID (200 train spikes) denoised data GMM score: 49.9818882433913
max gmm score: 49.72548301103939
min gmm score: 44.87138629207036
50 run gmm mean score: 48.07065613084811
50 run gmm std-dev score: 1.503227640182234
DY009 ID (200 train spikes) denoised data GMM score: 48.07065613084811
max gmm score: 44.28033538250909
min gmm score: 31.39884573758237
50 run gmm mean score: 39.61758101381981
50 run gmm std-dev score: 2.7179886027734343
test set A (DY016) (200 train spikes) ID data denoised GMM score: 39.61758101381981


In [7]:
dy016_ood_score = pca_gmm_average(dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 ID (200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_5c_memory_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_labels)
print("DY009 ID (200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (DY016) (200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 44.016484964451244
min gmm score: 34.27168700179398
50 run gmm mean score: 38.60536750922742
50 run gmm std-dev score: 2.2499194442618533
DY016 OOD data GMM score: 38.60536750922742
max gmm score: 43.2123786410486
min gmm score: 32.315591872083886
50 run gmm mean score: 39.17478255524215
50 run gmm std-dev score: 2.2927240799944033
DY009 OOD data GMM score: 39.17478255524215
max gmm score: 54.85513270593066
min gmm score: 36.60393970101378
50 run gmm mean score: 44.16644158201513
50 run gmm std-dev score: 3.601146433041381
test set A data GMM score: 44.16644158201513


In [8]:
dy016_ood_score = pca_gmm_average(dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 ID (200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_labels)
print("DY009 ID (200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (DY016) (200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 41.339826356672376
min gmm score: 29.16540999756477
50 run gmm mean score: 36.14958006262585
50 run gmm std-dev score: 2.557504424154209
DY016 OOD data GMM score: 36.14958006262585
max gmm score: 40.11572258482997
min gmm score: 30.17548036088694
50 run gmm mean score: 36.8479892578336
50 run gmm std-dev score: 2.278244767294241
DY009 OOD data GMM score: 36.8479892578336
max gmm score: 47.95558042099082
min gmm score: 33.61906273944858
50 run gmm mean score: 41.315006775805585
50 run gmm std-dev score: 3.819610125152901
test set A data GMM score: 41.315006775805585


In [12]:
dy016_ood_score = pca_gmm_average(dy016_id_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (test set A 1200 spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy016_id_5c_memory_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_labels)
print("DY009 OOD (test set A 1200 spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(dy016_id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (test set A 1200 spikes) data GMM score: " + str(testset_A_score))

max gmm score: 24.715127816076123
min gmm score: 20.984923304102924
50 run gmm mean score: 23.58112385071734
50 run gmm std-dev score: 0.5275223408566994
DY016 OOD (test set A 1200 spikes) data GMM score: 23.58112385071734
max gmm score: 21.238489666225092
min gmm score: 14.593883750991893
50 run gmm mean score: 17.9354498400044
50 run gmm std-dev score: 0.9226892008696899
DY009 OOD (test set A 1200 spikes) data GMM score: 17.9354498400044
max gmm score: 48.48341232339838
min gmm score: 42.088143716207256
50 run gmm mean score: 46.35456126187191
50 run gmm std-dev score: 2.02502218740775
test set A (test set A 1200 spikes) data GMM score: 46.35456126187191


In [13]:
dy016_ood_score = pca_gmm_average(dy016_id_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (test set A 1200 spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy016_id_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_labels)
print("DY009 OOD (test set A 1200 spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(dy016_id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (test set A 1200 spikes) data GMM score: " + str(testset_A_score))

max gmm score: 23.662534858583182
min gmm score: 10.746395164699681
50 run gmm mean score: 16.4852522192692
50 run gmm std-dev score: 3.3836952638107967
DY016 OOD (test set A 1200 spikes) data GMM score: 16.4852522192692
max gmm score: 22.049626883106228
min gmm score: 9.837814120694349
50 run gmm mean score: 15.901064382157278
50 run gmm std-dev score: 4.077840670932784
DY009 OOD (test set A 1200 spikes) data GMM score: 15.901064382157278
max gmm score: 43.13007028785448
min gmm score: 28.370391755074348
50 run gmm mean score: 36.29547030978494
50 run gmm std-dev score: 3.6242172401098993
test set A (test set A 1200 spikes) data GMM score: 36.29547030978494
